<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive

In [28]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
Data_basic = pd.DataFrame()

ValueError: ignored

In [5]:
Data_basic = pd.DataFrame(columns=['GameID','season','types','week','HomeTeam_ID','AwayTeam_ID','Winner'])
week = list(range(1, 19))
seasons=list(range(2017,2023))

In [6]:
seasons

[2017, 2018, 2019, 2020, 2021, 2022]

In [7]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [8]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [29]:
Data_basic.to_csv('/content/drive/MyDrive/play_ID_data.csv', index=False)


In [10]:
type(Data_basic['HomeTeam_ID'][0])





str

In [11]:
GameID='400951566'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])

In [12]:
drive_df['result']


0              TD
1          FUMBLE
2           DOWNS
3              TD
4              FG
5            PUNT
6              TD
7            PUNT
8            PUNT
9              TD
10    END OF HALF
11           PUNT
12           PUNT
13             TD
14           PUNT
15           PUNT
16             TD
17           PUNT
18             FG
19             TD
20          DOWNS
21           PUNT
22           PUNT
23             TD
24           PUNT
Name: result, dtype: object

In [13]:
for i in drive_df:
  print(i)

$ref
id
description
sequenceNumber
yards
isScore
offensivePlays
result
shortDisplayResult
displayResult
team.$ref
endTeam.$ref
start.period.type
start.period.number
start.clock.value
start.clock.displayValue
start.yardLine
start.text
end.period.type
end.period.number
end.clock.value
end.clock.displayValue
end.yardLine
end.text
timeElapsed.value
timeElapsed.displayValue
source.id
source.description
plays.$ref
plays.count
plays.pageIndex
plays.pageSize
plays.pageCount
plays.items


In [14]:
Hometeam=Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']

In [15]:
Hometeam[0]

'17'

In [40]:

Pass_Incompletion_Home=0
Punt_Home=0
Timeout_Home=0
Punt_Away=0
Pass_Incompletion_Away=0
Timeout_Away=0
homeScore=0
awayScore=0
PassReception_Home=0
PassReception_Away=0
Penalty_Home=0
Penalty_Away=0
for index, row in drive_df.iterrows():
  if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'][0])&(row['start.period.number']==2):
    for detail in range(len(row['plays.items'])):
      if row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
        Pass_Incompletion_Home=Pass_Incompletion_Home+1
      elif row['plays.items'][detail]['type']['text'] == 'Punt':
        Punt_Home=Punt_Home+1
      elif row['plays.items'][detail]['type']['text'] == 'Timeout':
        Timeout_Home=Timeout_Home+1
      elif row['plays.items'][detail]['type']['text'] == 'Pass Reception':
        PassReception_Home=PassReception_Home+1
      elif row['plays.items'][detail]['type']['text'] == 'Penalty':
        Penalty_Home=Penalty_Home+1
      homeScore=row['plays.items'][detail]['homeScore']
  elif re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'][0]:
    for detail in range(len(row['plays.items'])):
      if row['plays.items'][detail]['type']['text'] == 'Pass Incompletion':
        Pass_Incompletion_Away=Pass_Incompletion_Away+1
      elif row['plays.items'][detail]['type']['text'] == 'Punt':
        Punt_Away=Punt_Away+1
      elif row['plays.items'][detail]['type']['text'] == 'Timeout':
        Timeout_Away=Timeout_Away+1  
      elif row['plays.items'][detail]['type']['text'] == 'Pass Reception':
        PassReception_Away=PassReception_Away+1
      elif row['plays.items'][detail]['type']['text'] == 'Penalty':
        Penalty_Away=Penalty_Away+1
      awayScore=row['plays.items'][detail]['awayScore']
print(Timeout_Home)
print(Timeout_Away)
print(homeScore)
print(awayScore)
print(Penalty_Home)
print(Penalty_Away)



0
4
17
35
0
3


In [17]:
drive_df['plays.items'][3][2]['type']['text']

'Pass Reception'

In [18]:
homescore=0
awayScore=0
Pass_Incompletion_Home=0
Punt_Home=0
Time_Out=0

awayScore=0
for plays in range(len(drive_df['plays.items'])):
  for detail in range(len(drive_df['plays.items'][plays])):
    if drive_df['plays.items'][plays][detail]['type']['text'] == 'Pass Incompletion':
      Pass_Incompletion_Home=Pass_Incompletion_Home+1
    elif drive_df['plays.items'][plays][detail]['type']['text'] == 'Punt':
      Punt_Home=Punt_Home+1
    elif drive_df['plays.items'][plays][detail]['type']['text'] == 'Timeout':
      Time_Out=Time_Out+1
    print(drive_df['plays.items'][plays][detail]['type']['text'])
    #print(drive_df['plays.items'][plays][detail]['statYardage'])
    #print(drive_df['plays.items'][plays][detail]['scoreValue'])
    #print(drive_df['plays.items'][plays][detail]['awayScore'])
    #print(drive_df['plays.items'][plays][detail]['homeScore'])
print(Time_Out)

Kickoff
Pass Incompletion
Pass Reception
Rush
Rush
Pass Reception
Rush
Pass Reception
Pass Incompletion
Rush
Pass Incompletion
Rushing Touchdown
Kickoff
Fumble Recovery (Opponent)
Pass Incompletion
Pass Reception
Rush
Pass Incompletion
Rush
Pass Incompletion
Pass Reception
Rush
Rush
Rush
Pass Reception
Pass Reception
Pass Reception
Rush
Pass Reception
Pass Reception
Pass Reception
Pass Incompletion
Rush
Passing Touchdown
Kickoff
Rush
Pass Reception
Pass Reception
Rush
Rush
Pass Incompletion
Rush
Rush
End Period
Field Goal Good
Kickoff
Pass Incompletion
Rush
Sack
Punt
Rush
Rush
Pass Reception
Pass Incompletion
Rush
Pass Reception
Rush
Rush
Rush
Rush
Pass Incompletion
Rushing Touchdown
Kickoff
Rush
Rush
Rush
Rush
Sack
Punt
Rush
Pass Incompletion
Pass Incompletion
Punt
Rush
Rush
Timeout
Pass Reception
Two-minute warning
Pass Reception
Pass Reception
Rush
Pass Incompletion
Pass Reception
Timeout
Pass Reception
Pass Reception
Pass Reception
Pass Incompletion
Pass Reception
Timeout
Passing T

In [19]:
print(Pass_Incompletion_Home)

32


In [20]:
drive_df['plays.items']

0     [{'$ref': 'http://sports.core.api.espn.com/v2/...
1     [{'$ref': 'http://sports.core.api.espn.com/v2/...
2     [{'$ref': 'http://sports.core.api.espn.com/v2/...
3     [{'$ref': 'http://sports.core.api.espn.com/v2/...
4     [{'$ref': 'http://sports.core.api.espn.com/v2/...
5     [{'$ref': 'http://sports.core.api.espn.com/v2/...
6     [{'$ref': 'http://sports.core.api.espn.com/v2/...
7     [{'$ref': 'http://sports.core.api.espn.com/v2/...
8     [{'$ref': 'http://sports.core.api.espn.com/v2/...
9     [{'$ref': 'http://sports.core.api.espn.com/v2/...
10    [{'$ref': 'http://sports.core.api.espn.com/v2/...
11    [{'$ref': 'http://sports.core.api.espn.com/v2/...
12    [{'$ref': 'http://sports.core.api.espn.com/v2/...
13    [{'$ref': 'http://sports.core.api.espn.com/v2/...
14    [{'$ref': 'http://sports.core.api.espn.com/v2/...
15    [{'$ref': 'http://sports.core.api.espn.com/v2/...
16    [{'$ref': 'http://sports.core.api.espn.com/v2/...
17    [{'$ref': 'http://sports.core.api.espn.com

In [21]:
drive_df

,$ref,id,description,sequenceNumber,yards,isScore,offensivePlays,result,shortDisplayResult,displayResult,...,timeElapsed.value,timeElapsed.displayValue,source.id,source.description,plays.$ref,plays.count,plays.pageIndex,plays.pageSize,plays.pageCount,plays.items
0,http://sports.core.api.espn.com/v2/sports/foot...,4009515661,"9 plays, 73 yards, 2:52",1,73,True,9,TD,TD,Touchdown,...,172.0,2:52,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,12,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
1,http://sports.core.api.espn.com/v2/sports/foot...,4009515662,"1 play, 7 yards, 0:08",2,7,False,1,FUMBLE,FUMBLE,Fumble,...,8.0,0:08,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,2,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
2,http://sports.core.api.espn.com/v2/sports/foot...,4009515663,"7 plays, 22 yards, 2:35",3,22,False,7,DOWNS,DOWNS,Downs,...,155.0,2:35,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,8,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
3,http://sports.core.api.espn.com/v2/sports/foot...,4009515664,"12 plays, 90 yards, 6:22",4,90,True,12,TD,TD,Touchdown,...,382.0,6:22,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,12,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
4,http://sports.core.api.espn.com/v2/sports/foot...,4009515665,"8 plays, 67 yards, 3:07",5,67,True,8,FG,FG,Field Goal,...,187.0,3:07,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,11,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
5,http://sports.core.api.espn.com/v2/sports/foot...,4009515666,"3 plays, 1 yard, 1:39",6,1,False,3,PUNT,PUNT,Punt,...,99.0,1:39,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,5,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
6,http://sports.core.api.espn.com/v2/sports/foot...,4009515667,"11 plays, 82 yards, 5:50",7,82,True,11,TD,TD,Touchdown,...,350.0,5:50,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,12,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
7,http://sports.core.api.espn.com/v2/sports/foot...,4009515668,"5 plays, 6 yards, 3:43",8,6,False,5,PUNT,PUNT,Punt,...,223.0,3:43,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,7,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
8,http://sports.core.api.espn.com/v2/sports/foot...,4009515669,"3 plays, 4 yards, 0:57",9,4,False,3,PUNT,PUNT,Punt,...,57.0,0:57,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,4,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
9,http://sports.core.api.espn.com/v2/sports/foot...,40095156610,"12 plays, 92 yards, 2:34",10,92,True,12,TD,TD,Touchdown,...,154.0,2:34,2,Feed,http://sports.core.api.espn.com/v2/sports/foot...,18,1,1000,1,[{'$ref': 'http://sports.core.api.espn.com/v2/...
